In [ ]:
# %load_ext autoreload
# %autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['figure.titlesize'] = 30
mpl.rcParams.update({"axes.grid" : True})

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
cfgList=[]
filesDic={}
files2ptKey={}
files3ptKey={}

data={}

def loadData(basePath):
    f={}
    for key in filesDic.keys():
        f[key]={}
        for file in filesDic[key]:
            f[key][file] = h5py.File(basePath + file + '.h5','r')

    for key in files2ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for projg in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][projg]={}
                        for cont in tF[projAStr][projBStr][projg][cfgList[0]].keys():
                            data[key][file][projA][projB][projg][cont]=np.array([
                                f[key][file][projAStr][projBStr][projg][cfg][cont][()] for cfg in cfgList
                                ])

    for key in files3ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for projg in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][projg]={}
                        for cont in tF[projAStr][projBStr][projg][cfgList[0]].keys():
                            data[key][file][projA][projB][projg][cont]=np.array([
                                f[key][file][projAStr][projBStr][projg][cfg][cont][()] for cfg in cfgList
                                ])

    for key in filesDic.keys():
        for file in filesDic[key]:
            f[key][file].close()

path_projData='/project/s1174/lyan/code/projectData/'

basePath_NJN=path_projData+'nucleon_sigma_term/cA211.53.24/NJN/proj/'
basePath_NJNpi=path_projData+'nucleon_sigma_term/cA211.53.24/NJNpi_GEVP/proj/'
basePath_loop=path_projData+'nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion/data_post_merged/'

with h5py.File(basePath_NJN+'NJN.h5') as f:
    cfgList_NJN=list(yu.deepKey(f,3).keys())

with h5py.File(basePath_NJNpi+'N_2pt.h5') as f:
    cfgList_NJNpi=list(yu.deepKey(f,3).keys())

cfgList=list( set(cfgList_NJN) & set(cfgList_NJNpi) )
# cfgList=cfgList_NJNpi

cfgList.sort()
Ncfg=len(cfgList)
print(Ncfg)


filesDic={}
files2ptKey=[]
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['NJN']
filesDic['NJNpi']=['NJN-pi0i']
loadData(basePath_NJN)

filesDic={}
files2ptKey=['NN','NNpi','NpiN','NpiNpi']
filesDic['NN']=['N_2pt']
filesDic['NNpi']=['T','N_2pt-pi0i']
filesDic['NpiN']=['D1ii','N_2pt-pi0f']
filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt','N_2pt-pi0f-pi0i','T-pi0f','D1ii-pi0i']
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['N_2pt-j','N_2pt-j_50']
filesDic['NJNpi']=['B','W','Z','N_2pt-j-pi0i','N_2pt-j&pi0i','T-j']
loadData(basePath_NJNpi)

# filesDic={}
# files2ptKey=['NN','NNpi','NpiN','NpiNpi']
# filesDic['NN']=['N_2pt']
# filesDic['NNpi']=['T']
# filesDic['NpiN']=['D1ii']
# filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
# files3ptKey=['NJNpi']
# filesDic['NJNpi']=['B','W','Z']
# loadData(basePath_NJNpi)

data['others']={}
with h5py.File(basePath_loop+'insertLoop.h5') as f:
    data['others']['u+d_id_vev']=np.array([f[cfg]['u+d_id_vev'][()] for cfg in cfgList])
    data['others']['u-d_g5_vev']=np.array([f[cfg]['u-d_g5_vev'][()] for cfg in cfgList])
with h5py.File(basePath_loop+'pi0Loop.h5') as f:
    data['others']['pi0Loop_g5_vev']=np.array([f[cfg]['pi0Loop_g5_vev'][()] for cfg in cfgList])

In [ ]:
test_NNpi=1; test_NpiNpi=1; test_NJN=1; test_NJNpi=1; noD1ii=False

diags_subtract={'N_2pt-j_50'}

diags_key={}
diags_all=set(); diags_pi0Loop=set(); diags_jLoop=set()
for key in data.keys():
    if key in ['others']:
        continue
    diags_key[key] = set(data[key])
    diags_all = diags_all | diags_key[key] - diags_subtract
    for diag in data[key]:
        t=diag.split('-')
        if 'pi0i' in t or 'pi0f' in t:
            diags_pi0Loop.add(diag)
        if 'j' in t:
            diags_jLoop.add(diag)

diags_pi0Loop = diags_pi0Loop - diags_subtract
diags_jLoop = diags_jLoop - diags_subtract

diags_loopful = diags_pi0Loop | diags_jLoop
diags_loopless = diags_all - diags_loopful
diags_jLoopless = diags_all - diags_jLoop
diags_all_2pt = diags_key['NN'] | diags_key['NNpi'] | diags_key['NpiNpi']
diags_all_3pt = diags_all - diags_all_2pt

projgs_P012=['P0','P01','P02']
projgs_Pz12=['Pz','P01','P02']

Ntime=yu.deepKey(data['NN'],5).shape[1]

def getDat2pt(opa,opb,projg,diags):
    (tpa,proja)=opa; (tpb,projb)=opb

    if tpa in ['12']:
        opa1=('n&pi+',proja); opa2=('p&pi0',proja)
        return np.sqrt(2/3)*getDat2pt(opa1,opb,projg,diags)-np.sqrt(1/3)*getDat2pt(opa2,opb,projg,diags)
    if tpb in ['12']:
        opb1=('n&pi+',projb); opb2=('p&pi0',projb)
        return np.sqrt(2/3)*getDat2pt(opa,opb1,projg,diags)-np.sqrt(1/3)*getDat2pt(opa,opb2,projg,diags)

    if tpa in ['p'] and tpb in ['p']:
        key='NN'
        return data[key]['N_2pt'][proja][projb][projg][tpa+'_'+tpb]+np.zeros([Ncfg,Ntime])
    elif tpa in ['p']:
        key='NNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['T']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0i']:
                return [tpa+'_p'] if tpb =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])
    elif tpb in ['p']:
        key='NpiN'
        t_diags = diags & diags_key[key]
        if noD1ii and 'D1ii' in t_diags:
            t=np.conj(getDat2pt(opb,opa,projg,{'T'}))
            t_diags = t_diags - {'D1ii'}
        else:
            t=0
        def get_conts(diag):
            if diag in ['D1ii']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0f']:
                return ['p_'+tpb] if tpa =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return np.conj(test_NNpi)*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])+t
    else:
        key='NpiNpi'
        t_diags = diags & diags_key[key]
        if noD1ii and 'D1ii-pi0i' in t_diags:
            t=np.conj(getDat2pt(opb,opa,projg,{'T-pi0f'}))
            t_diags = t_diags - {'D1ii-pi0i'}
        else:
            t=0
        def get_conts(diag):
            if diag in ['B_2pt','W_2pt','Z_2pt']:
                return [tpa+'_'+tpb]
            if diag in ['M_correct_2pt']:
                return [tpa+'_'+tpa] if tpa==tpb else []
            if diag in ['N_2pt-pi0f-pi0i']:
                return ['p_p'] if tpa==tpb=='p&pi0' else []
            if diag in ['T-pi0f']:
                return ['p_'+tpb] if tpa=='p&pi0' else []
            if diag in ['D1ii-pi0i']:
                return [tpa+'_p'] if tpb=='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NpiNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])+t

def getDat2ptMat(ops,projg,diags):
    t=np.transpose([[getDat2pt(opa,opb,projg,diags) for opb in ops] for opa in ops],[2,3,0,1])
    t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def getDat3pt(opa,opb,Gc,iso,projg,tf,diags):
    (tpa,proja)=opa; (tpb,projb)=opb

    if tpa in ['12']:
        opa1=('n&pi+',proja); opa2=('p&pi0',proja)
        return np.sqrt(2/3)*getDat3pt(opa1,opb,Gc,iso,projg,tf,diags)-np.sqrt(1/3)*getDat3pt(opa2,opb,Gc,iso,projg,tf,diags)
    if tpb in ['12']:
        opb1=('n&pi+',projb); opb2=('p&pi0',projb)
        return np.sqrt(2/3)*getDat3pt(opa,opb1,Gc,iso,projg,tf,diags)-np.sqrt(1/3)*getDat3pt(opa,opb2,Gc,iso,projg,tf,diags)

    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]

    if tpa in ['p'] and tpb in ['p']:
        key='NJN'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['NJN','N_2pt-j','N_2pt-j_50']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont) for diag in t_diags for cont in get_conts(diag)]
        return test_NJN*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])       
    elif tpa in ['p']:
        key='NJNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['B','W','Z']:
                return [tpa+'_j'+iso+'_'+tpb+'_deltat_'+str(tf)]
            if diag in ['N_2pt-j&pi0i','N_2pt-j-pi0i','NJN-pi0i']:
                return [tpa+'_p'+'_j'+iso+'_deltat_'+str(tf)] if tpb == 'p&pi0' else []
            if diag in ['T-j']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NJNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])
    elif tpb in ['p']:
        if projb[0] != (0,0,0):
            return np.zeros([Ncfg,tf-1])
        return np.conj(getDat3pt(opb,opa,Gc,iso,projg,tf,diags)[:,::-1]) * (1 if Gc in ['id','gt'] else (-1))
    else:
        return np.zeros([Ncfg,tf-1])
    
def getDat3ptMat(opas,opbs,Gc,iso,projg,tf,diags):
    t=np.transpose([[getDat3pt(opa,opb,Gc,iso,projg,tf,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    return t

tfList=[10,12,14]
def getDat(opas,opbs,Gc,iso,projg,diags):
    pta=opas[0][1][0]; ptb=opbs[0][1][0]
    assert(pta==(0,0,0)); pc=ptb
    if pc==(0,0,0) and projg=='Pzt':
        projg='Pz'
    if projg in ['Pz','Pzt']:
        projg_2pt='P0'
    else:
        projg_2pt=projg
    dat2ptA=getDat2ptMat(opas,projg_2pt,diags)
    dat2ptB=getDat2ptMat(opbs,projg_2pt,diags)
    dat3pt={}
    for tf in tfList:
        dat3pt[tf]=getDat3ptMat(opas,opbs,Gc,iso,projg,tf,diags)
    if (Gc,iso) == ('id','+'):
        dat_jLoop=data['others']['u+d_id_vev']
    elif (Gc,iso) == ('g5','-'):
        dat_jLoop=data['others']['u-d_g5_vev']
    else:
        dat_jLoop=np.zeros(Ncfg)

    t=np.array([
        np.array([dat2ptA[cfg],dat2ptB[cfg]]+[np.array([dat3pt[tf][cfg] for tf in tfList],dtype=object)]+[dat_jLoop[cfg]],dtype=object)
    for cfg in range(Ncfg)], dtype=object)
    return t


pt0=(0,0,0); pt1=(0,0,1); pt2=(0,0,-1); pt3=(1,0,0)
tp0='p'; tp1='n&pi+'; tp2='p&pi0'; tp3='12'
op00=(tp0,(pt0,'a')); op01=(tp1,(pt0,('N1\\pi1','a'))); op02=(tp2,(pt0,('N1\\pi1','a'))); op03=(tp3,(pt0,('N1\\pi1','a')))

op10=(tp0,(pt1,'a')); op11=(tp1,(pt1,('N1\\pi0','a'))); op12=(tp2,(pt1,('N1\\pi0','a'))); op13=(tp3,(pt1,('N1\\pi0','a')))
op101=(tp0,(pt1,'a')); op111=(tp1,(pt1,('N0\\pi1','a'))); op121=(tp2,(pt1,('N0\\pi1','a'))); op131=(tp3,(pt1,('N0\\pi1','a')))
op102=(tp0,(pt1,'a')); op112=(tp1,(pt1,('N2\\pi1','a'))); op122=(tp2,(pt1,('N2\\pi1','a'))); op132=(tp3,(pt1,('N2\\pi1','a')))
op103=(tp0,(pt1,'a')); op113=(tp1,(pt1,('N2\\pi1','b'))); op123=(tp2,(pt1,('N2\\pi1','b'))); op133=(tp3,(pt1,('N2\\pi1','b')))

op20=(tp0,(pt2,'a')); op21=(tp1,(pt2,('N1\\pi0','a'))); op22=(tp2,(pt2,('N1\\pi0','a'))); op23=(tp3,(pt2,('N1\\pi0','a')))
op201=(tp0,(pt2,'a')); op211=(tp1,(pt2,('N0\\pi1','a'))); op221=(tp2,(pt2,('N0\\pi1','a'))); op231=(tp3,(pt2,('N0\\pi1','a')))

op30=(tp0,(pt3,'a')); op31=(tp1,(pt3,('N1\\pi0','a'))); op32=(tp2,(pt3,('N1\\pi0','a'))); op33=(tp3,(pt2,('N1\\pi0','a')))
op301=(tp0,(pt3,'a')); op311=(tp1,(pt3,('N0\\pi1','a'))); op321=(tp2,(pt3,('N0\\pi1','a'))); op331=(tp3,(pt2,('N0\\pi1','a')))

In [ ]:
def run2pt(opa,opb,diags=diags_all,title=None,ylim1=None,ylim2=[.4,1.1],preFactor=1,axs=None,avgConjFlag=False,xshift=0,projgs=['P01','P0','P02'],color=None,fmt=None,label=None):
    tmin=1; tmax=16

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        title=opa[0]+'_'+opb[0]+(': '+title if title!=None else '')
        fig.suptitle(title)

    for projg in projgs:
        colorDic={'P01':'r','P02':'b','P0':'g'}
        if color == None:
            color=colorDic[projg]
        
        dat=preFactor*getDat2pt(opa,opb,projg,diags)
        if avgConjFlag:
            dat=preFactor*(getDat2pt(opa,opb,projg,diags)+np.conj(getDat2pt(opb,opa,projg,diags)))/2

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        # print(tMean,tErr)
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=fmt,label=label)
        axs[irow,icol].set_xticks(range(0,tmax,1))  
        if ylim1=='sym':
            t=max(abs(np.array(axs[irow,icol].get_ylim())))
            axs[irow,icol].set_ylim([-t,t])
        else:
            axs[irow,icol].set_ylim(ylim1)
        

        # irow=0; icol=1
        # def tFunc(dat):
        #     t=np.mean(dat,axis=0)
        #     t=np.real(t)
        #     t=np.log(t/np.roll(t,-1,axis=0))
        #     return t
        # (mean,cov,err) = yu.jackknife(dat,tFunc)
        # tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        # axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt='s',elinewidth=2,capsize=6,mfc='white')
        # axs[irow,icol].set_xticks(range(0,tmax,1))
        # axs[irow,icol].set_ylim(ylim2)
        # if opa[1][0] == (0,0,0):
        #     axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        # elif np.linalg.norm(opa[1][0]) == 1.0:
        #     axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 
        #     axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan') 
        #     axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange')        

In [ ]:
(opx,opy,opz)=(op00,op01,op02)
# (opx,opy,opz)=(op10,op11,op12)
# (opx,opy,opz)=(op10,op111,op121)

# N-Npi: test isospin 
projgs=['P0']
fig,axs=yu.getFigAxs(1,1,8,10); #fig.suptitle('imag')
(opa,opb)=(opx,opy)
run2pt(opa,opb,preFactor=-1j,axs=axs,projgs=projgs,color='r',fmt='s',xshift=-0.3,label=r'$p\to n\,\pi^+$')
(opa,opb)=(opx,opz)
run2pt(opa,opb,diags=diags_loopless,preFactor=-1j*(-1*np.sqrt(2)),axs=axs,xshift=-0.,avgConjFlag=False,projgs=projgs,color='g',fmt='d',label=r'$p\to p\,\pi^0$: conn')
run2pt(opa,opb,preFactor=-1j*(-1*np.sqrt(2)),axs=axs,xshift=0.3,avgConjFlag=True,projgs=projgs,color='b',fmt='o',label=r'$p\to p\,\pi^0$: full') # ylim1=[-1e-10,0]

axs[0,0].set_xlabel(r'$t_{sink}$')
axs[0,0].set_ylabel(r'correlation functions')
axs[0,0].legend();

In [ ]:
def run2pt(opa,opb,diags=diags_all,title=None,ylim1=None,ylim2=[.4,1.1],preFactor=1,axs=None,avgConjFlag=False,xshift=0,projgs=['P01','P0','P02']):
    tmin=1; tmax=15

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        title=opa[0]+'_'+opb[0]+(': '+title if title!=None else '')
        fig.suptitle(title)

    for projg in projgs:
        colorDic={'P01':'r','P02':'b','P0':'g'}
        
        dat=preFactor*getDat2pt(opa,opb,projg,diags)
        if avgConjFlag:
            dat=preFactor*(getDat2pt(opa,opb,projg,diags)+np.conj(getDat2pt(opb,opa,projg,diags)))/2

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=colorDic[projg],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,1))  
        if ylim1=='sym':
            t=max(abs(np.array(axs[irow,icol].get_ylim())))
            axs[irow,icol].set_ylim([-t,t])
        else:
            axs[irow,icol].set_ylim(ylim1)
        

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            t=np.log(t/np.roll(t,-1,axis=0))
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=colorDic[projg],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,1))
        axs[irow,icol].set_ylim(ylim2)
        if opa[1][0] == (0,0,0):
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        elif np.linalg.norm(opa[1][0]) == 1.0:
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 
            axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan') 
            axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange')        

In [ ]:
(opx,opy,opz)=(op00,op01,op02)
(opx,opy,opz)=(op10,op11,op12)
# (opx,opy,opz)=(op10,op111,op121)
diags={'N_2pt-pi0i','N_2pt-pi0f'}
(opa,opb)=(opx,opz)
# run2pt(opa,opb,preFactor=-1j,xshift=0.5,avgConjFlag=False,projgs=['P01'],diags={'T'})
run2pt(opa,opb,preFactor=1,xshift=0.5,avgConjFlag=False,projgs=['P01'],diags=diags)
# (opa,opb)=(opz,opx)
run2pt(opa,opb,preFactor=1,xshift=0.5,avgConjFlag=False,projgs=['P01'],diags=diags)
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,preFactor=-1j*(-1*np.sqrt(2)),xshift=0.5,avgConjFlag=True,projgs=['P01'],diags=diags)

In [ ]:
def run2pt(opa,opb,diags=diags_all,title=None,ylim1=None,ylim2=[.4,1.1],preFactor=1,axs=None,avgConjFlag=False,xshift=0,projgs=['P01','P0','P02'],color=None,fmt=None,label=None):
    tmin=1; tmax=16

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        title=opa[0]+'_'+opb[0]+(': '+title if title!=None else '')
        fig.suptitle(title)

    for projg in projgs:
        colorDic={'P01':'r','P02':'b','P0':'g'}
        if color == None:
            color=colorDic[projg]
        
        dat=preFactor*getDat2pt(opa,opb,projg,diags)
        if avgConjFlag:
            dat=preFactor*(getDat2pt(opa,opb,projg,diags)+np.conj(getDat2pt(opb,opa,projg,diags)))/2

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        # print(tMean,tErr)
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=fmt,label=label)
        axs[irow,icol].set_xticks(range(0,tmax,1))  
        if ylim1=='sym':
            t=max(abs(np.array(axs[irow,icol].get_ylim())))
            axs[irow,icol].set_ylim([-t,t])
        else:
            axs[irow,icol].set_ylim(ylim1)
        

        # irow=0; icol=1
        # def tFunc(dat):
        #     t=np.mean(dat,axis=0)
        #     t=np.real(t)
        #     t=np.log(t/np.roll(t,-1,axis=0))
        #     return t
        # (mean,cov,err) = yu.jackknife(dat,tFunc)
        # tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        # axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt='s',elinewidth=2,capsize=6,mfc='white')
        # axs[irow,icol].set_xticks(range(0,tmax,1))
        # axs[irow,icol].set_ylim(ylim2)
        # if opa[1][0] == (0,0,0):
        #     axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        # elif np.linalg.norm(opa[1][0]) == 1.0:
        #     axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 
        #     axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan') 
        #     axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange')        

In [ ]:
(opx,opy,opz)=(op00,op01,op02)
(opx,opy,opz)=(op10,op11,op12)
(opx,opy,opz)=(op10,op111,op121)


projgs=['P0']; diags=diags_loopless
fig,axs=yu.getFigAxs(1,1,8,10); axs[0,0].set_ylim([0,0.001e-7]); #fig.suptitle('title')
(opa,opb)=(opy,opy)
run2pt(opa,opb,diags,preFactor=1,axs=axs,xshift=-0.3,color='r',fmt='s',label=r'$n\,\pi^+\to n\,\pi^+$',projgs=projgs)
(opa,opb)=(opy,opz)
run2pt(opa,opb,diags,preFactor=(-1*np.sqrt(2)),axs=axs,xshift=0.,color='g',fmt='d',label=r'$n\,\pi^+\to p\,\pi^0$',projgs=projgs)
(opa,opb)=(opz,opz)
run2pt(opa,opb,diags,preFactor=2,axs=axs,xshift=0.3,color='b',fmt='o',label=r'$p\,\pi^0\to p\,\pi^0$',projgs=projgs)
axs[0,0].set_xlabel(r'$t_{sink}$')
axs[0,0].set_ylabel(r'correlation functions')
axs[0,0].legend()

projgs=['P0']; diags=diags_all
fig,axs=yu.getFigAxs(1,1,8,10); axs[0,0].set_ylim([0,0.001e-7]); #fig.suptitle('title')
(opa,opb)=(opy,opy)
run2pt(opa,opb,diags,preFactor=1,axs=axs,xshift=-0.3,color='r',fmt='s',label=r'$n\,\pi^+\to n\,\pi^+$',projgs=projgs)
(opa,opb)=(opy,opz)
run2pt(opa,opb,diags,preFactor=(-1*np.sqrt(2)),axs=axs,xshift=0.,color='g',fmt='d',label=r'$n\,\pi^+\to p\,\pi^0$',projgs=projgs)
(opa,opb)=(opz,opz)
run2pt(opa,opb,diags,preFactor=2,axs=axs,xshift=0.3,color='b',fmt='o',label=r'$p\,\pi^0\to p\,\pi^0$',projgs=projgs)
axs[0,0].set_xlabel(r'$t_{sink}$')
# axs[0,0].set_ylabel(r'correlation functions')
axs[0,0].legend()

In [ ]:
def run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None,color='r',xshift=0,mfc=None):
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+(': '+title if title!=None else ''))

    # yu.addColHeader(axs,['3pt'])

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        x_shift += xshift
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        dat=getDat([opa],[opb],Gc,iso,projg,diags)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.array([np.real(preFactor*flip*(t[2][i_tf][:,0,0])) for i_tf in range(len(tfList))],dtype=object)
            return t

        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            # print(i_tf,tMean,tErr)
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],mfc=mfc)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)

        # irow=0; icol=1
        # def tFunc(dat):
        #     t=np.mean(dat,axis=0)
            
        #     def getRes(i_tf):
        #         tf=tfList[i_tf]
        #         nom=t[2][i_tf][:,0,0]
        #         denom=np.sqrt( np.real(t[0][tf]) * np.real(t[1][tf]) *\
        #         np.real(t[0][1:tf])[::-1]/np.real(t[0][1:tf]) *\
        #         np.real(t[1][1:tf]/np.real(t[1][1:tf])[::-1]) )[:,0,0]
        #         return np.real(preFactor * (nom/denom-vevFactor*t[3]) )
        #     t=np.array([getRes(i_tf) for i_tf in range(len(tfList))],dtype=object)
        #     return t

        # (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        # colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        # fmtDic={10:'s',12:'d',14:'o'}

        # for i_tf in range(len(tfList)):
        #     tf=tfList[i_tf]
        #     tMean=mean[i_tf];tErr=err[i_tf]
        #     axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
        #         fmt=fmtDic[tf],color=colorDic[(0,0)],elinewidth=2,capsize=6,mfc='white')
        # axs[irow,icol].set_xticks(range(-6,6,2))
        # axs[irow,icol].set_ylim(ylim2)


In [ ]:
(opx,opy,opz,opn)=(op00,op01,op02,op03)
# (opx,opy,opz,opn)=(op10,op11,op12,op13)
# (opx,opy,opz,opn)=(op10,op111,op121,op131)
# (opx,opy,opz,opn)=(op20,op21,op22,op23)
# (opx,opy,opz,opn)=(op30,op31,op32,op33)

ratio=1j

(Gc,iso,projgs,preFactor)=('gt','-',['P0'],1)
# (Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j)
# (Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],1)
# (Gc,iso,projgs,preFactor)=('gt','+',['P0'],1)
# (Gc,iso,projgs,preFactor)=('id','+',['P0'],1)

fig,axs=yu.getFigAxs(1,1,8*0.8,10*0.8)

diags={'B','W','Z','NJN-pi0i', 'N_2pt-j&pi0i', 'N_2pt-j-pi0i', 'T-j'}
diags={'B','W','Z','NJN-pi0i', 'N_2pt-j&pi0i'}
# diags={'W','NJN-pi0i'}
# diags={'T-j'}
(opa,opb)=(op00,op00)
run3pt(opa,opb,Gc,iso,projgs,1,{'NJN','N_2pt-j'},axs=axs,color='g',xshift=0.2)